## CH6 EXERCISE 1

### Part A

In [22]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import random
import numpy as np


random.seed(1628)

def generate_random_dataset(num_instances, num_dimensions):
  X = np.random.rand(num_instances, num_dimensions)
  y = np.random.randint(0, 2, size=num_instances)
  return X, y

def compute_capacity(X, y, num_instances):
  num_removed = 0
  knn = KNeighborsClassifier(n_neighbors=1)
  for i in range(num_instances):
    X_copy = np.delete(X, i, axis=0)
    y_copy = np.delete(y, i)

    knn.fit(X_copy, y_copy)
    y_pred = knn.predict([X[i]])

    if (y_pred[0]==y[i]):
      num_removed += 1

  return (num_instances - num_removed)

def sample_multiple_times(d, n_full):
  iterations = 100
  total = 0
  for iter in range(iterations):
    X_train, y_train = generate_random_dataset(n_full, d)
    total += compute_capacity(X_train, y_train, n_full)
  n_avg = total / iterations
  result = n_full/n_avg
  print("d={}: n_full={}, Avg. req. points for memorization n_avg={:.2f}, n_full/n_avg={:.16f}".format(d, n_full, n_avg, result))


sample_multiple_times(2, 4)
sample_multiple_times(4, 16)
sample_multiple_times(8, 256)


d=2: n_full=4, Avg. req. points for memorization n_avg=2.17, n_full/n_avg=1.8433179723502304
d=4: n_full=16, Avg. req. points for memorization n_avg=8.39, n_full/n_avg=1.9070321811680571
d=8: n_full=256, Avg. req. points for memorization n_avg=128.76, n_full/n_avg=1.9881950916433677


d=2: n_full=4, Avg. req. points for memorization n_avg=2.17, n_full/n_avg=1.8433179723502304

d=4: n_full=16, Avg. req. points for memorization n_avg=8.39, n_full/n_avg=1.9070321811680571

d=8: n_full=256, Avg. req. points for memorization n_avg=128.76, n_full/n_avg=1.9881950916433677

Since n_full/n_avg is ~ 2 and approaches 2 as d increases, information limit of 2 bits per parameter holds for KNN.

### Part B

In [27]:
# For multiclass

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import random
import numpy as np


random.seed(1628)

def generate_random_dataset(num_instances, num_dimensions, num_classes):
  X = np.random.rand(num_instances, num_dimensions)
  y = np.random.randint(num_classes, size=num_instances)
  return X, y

def compute_capacity(X, y, num_instances):
  num_removed = 0
  knn = KNeighborsClassifier(n_neighbors=1)
  for i in range(num_instances):
    X_copy = np.delete(X, i, axis=0)
    y_copy = np.delete(y, i)

    knn.fit(X_copy, y_copy)
    y_pred = knn.predict([X[i]])

    if (y_pred[0]==y[i]):
      num_removed += 1
  return (num_instances - num_removed)

def sample_multiple_times(d, n_full, num_classes):
  iterations = 100
  total = 0
  for iter in range(iterations):
    X_train, y_train = generate_random_dataset(n_full, d, num_classes)
    total += compute_capacity(X_train, y_train, n_full)
  n_avg = total / iterations
  result = n_full/n_avg
  print("d={}: n_full={}, Avg. req. points for memorization n_avg={:.2f}, n_full/n_avg={:.16f}".format(d, n_full, n_avg, result, num_classes))

sample_multiple_times(2, 4, 8)
sample_multiple_times(4, 16, 8)
sample_multiple_times(8, 256, 8)


d=2: n_full=4, Avg. req. points for memorization n_avg=3.63, n_full/n_avg=1.1019283746556474
d=4: n_full=16, Avg. req. points for memorization n_avg=14.12, n_full/n_avg=1.1331444759206799
d=8: n_full=256, Avg. req. points for memorization n_avg=223.58, n_full/n_avg=1.1450040254047769


d=2: n_full=4, Avg. req. points for memorization n_avg=3.63, n_full/n_avg=1.1019283746556474

d=4: n_full=16, Avg. req. points for memorization n_avg=14.12, n_full/n_avg=1.1331444759206799

d=8: n_full=256, Avg. req. points for memorization n_avg=223.58, n_full/n_avg=1.1450040254047769

The expected n_full/n_avg is c/c-1 (compression). With 8 classes, you would expect it to approach 1.14 as d increases, which you can see happens here.

## CH6 EXERCISE 2

###Part A

In [174]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.tree import _tree
from sklearn.tree import export_text
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_wine

# Borrowed function to print decisions in a clean, human-friendly way, credit: https://mljar.com/blog/extract-rules-decision-tree/
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []

    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]

    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]

    rules = []
    for path in paths:
        rule = "if "

        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]

    return rules

In [175]:
# Initialize decision tree classifier and grid search for tuning

# Load and split dataset

data = load_breast_cancer()
breast_cancer_X = data.data
breast_cancer_y = data.target

def count_rules(X, y, depth, nodes, print_flag):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  classifier = DecisionTreeClassifier(max_depth = depth, max_leaf_nodes = nodes)
  model = classifier.fit(X_train, y_train)

  # predict on the test data
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  if(print_flag == True):
    print("Accuracy:", accuracy)
    print("Max tree depth:", model.get_depth())
    print("Num leaf nodes:", model.tree_.n_leaves)

  # Print out rules (follow tree branch until "class" -> 1 if/else statement)
  text_representation = tree.export_text(classifier)
  #print(text_representation)
  num_clauses_final = text_representation.count("<=") + text_representation.count(">")
  #print(str(num_clauses_final) + " if/else clauses")

  # Print out rules (and count of how many), formatted to be readable
  rules = get_rules(classifier, data.feature_names, data.target_names)
  #for r in rules:
  #    print(r)
  #print(str(len(rules)) + " decisions")
  if(print_flag == True):
    print("\nIf/then clauses: " + str(num_clauses_final) + "\nDecisions: " + str(len(rules)))
  return num_clauses_final, accuracy, len(rules)


In [176]:
# Original
num_clauses, accuracy, decisions = count_rules(breast_cancer_X, breast_cancer_y, None, None, True)
breast_cancer_original_accuracy = accuracy

Accuracy: 0.9385964912280702
Max tree depth: 7
Num leaf nodes: 16

If/then clauses: 30
Decisions: 16


In [178]:
# Tuning depth
current_min_clauses = 30
current_min_decisions = 16
for i in range(1,8):
  num_clauses, accuracy, decisions = count_rules(breast_cancer_X, breast_cancer_y, i, 16, False)
  if (accuracy >= breast_cancer_original_accuracy):
    if (num_clauses < current_min_clauses):
      current_min_clauses = num_clauses
      print("accuracy: "  + str(accuracy))
      print("clauses: " + str(num_clauses))
      print("depth: " + str(i))
      print('\n')
    if (decisions < current_min_decisions):
      current_min_decisions = decisions
      print("accuracy: "  + str(accuracy))
      print("decisions: " + str(decisions))
      print("depth: " + str(i))

accuracy: 0.9385964912280702
clauses: 14
depth: 3


accuracy: 0.9385964912280702
decisions: 8
depth: 3


In [177]:
# Tuning nodes
current_min_clauses = 30
current_min_decisions = 16
for i in range(2,17):
  num_clauses, accuracy, decisions = count_rules(breast_cancer_X, breast_cancer_y, 7, i, False)
  if (accuracy >= breast_cancer_original_accuracy):
    if (num_clauses < current_min_clauses):
      current_min_clauses = num_clauses
      print("accuracy: "  + str(accuracy))
      print("clauses: " + str(num_clauses))
      print("nodes: " + str(i))
      print('\n')
    if (decisions < current_min_decisions):
      current_min_decisions = decisions
      print("accuracy: "  + str(accuracy))
      print("decisions: " + str(decisions))
      print("nodes: " + str(i))

accuracy: 0.9473684210526315
clauses: 8
nodes: 5


accuracy: 0.9473684210526315
decisions: 5
nodes: 5


In [179]:
# Tune both (FINAL RESULT)
num_clauses, accuracy, decisions = count_rules(breast_cancer_X, breast_cancer_y, 3, 5, True)

Accuracy: 0.9473684210526315
Max tree depth: 3
Num leaf nodes: 5

If/then clauses: 8
Decisions: 5


In [180]:
# Minimum clauses (ignoring accuracy)
num_clauses, accuracy, decisions = count_rules(breast_cancer_X, breast_cancer_y, 1, 2, True)

Accuracy: 0.8947368421052632
Max tree depth: 1
Num leaf nodes: 2

If/then clauses: 2
Decisions: 2


### Part B

Another binary classification dataset: australian credit



In [ ]:
!pip install ucimlrepo

In [198]:
from sklearn.datasets import fetch_openml
australian_data = fetch_openml(name='Australian')
australian_X = australian_data.data
australian_y = australian_data.target

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [213]:
num_clauses, accuracy, decisions = count_rules(australian_X, australian_y, None, None, True)
australian_original_accuracy = accuracy
australian_original_decisions = decisions
australian_original_clauses = num_clauses

Accuracy: 0.8478260869565217
Max tree depth: 12
Num leaf nodes: 70

If/then clauses: 134
Decisions: 70


In [214]:
# Tuning depth
current_min_clauses = australian_original_clauses
current_min_decisions = australian_original_decisions
for i in range(1,13):
  num_clauses, accuracy, decisions = count_rules(australian_X, australian_y, i, 70, False)
  if (accuracy >= australian_original_accuracy):
    if (num_clauses < current_min_clauses):
      current_min_clauses = num_clauses
      print("accuracy: "  + str(accuracy))
      print("clauses: " + str(num_clauses))
      print("depth: " + str(i))
      print('\n')
    if (decisions < current_min_decisions):
      current_min_decisions = decisions
      print("accuracy: "  + str(accuracy))
      print("decisions: " + str(decisions))
      print("depth: " + str(i))

accuracy: 0.8478260869565217
clauses: 14
depth: 3


accuracy: 0.8478260869565217
decisions: 8
depth: 3


In [215]:
# Tuning nodes
current_min_clauses = australian_original_clauses
current_min_decisions = australian_original_decisions
for i in range(2,71):
  num_clauses, accuracy, decisions = count_rules(australian_X, australian_y, 12, i, False)
  if (accuracy >= australian_original_accuracy):
    if (num_clauses < current_min_clauses):
      current_min_clauses = num_clauses
      print("accuracy: "  + str(accuracy))
      print("clauses: " + str(num_clauses))
      print("nodes: " + str(i))
      print('\n')
    if (decisions < current_min_decisions):
      current_min_decisions = decisions
      print("accuracy: "  + str(accuracy))
      print("decisions: " + str(decisions))
      print("nodes: " + str(i))

accuracy: 0.8623188405797102
clauses: 8
nodes: 5


accuracy: 0.8623188405797102
decisions: 5
nodes: 5


In [224]:
# Best result (FINAL RESULT)
# Note: not using both best depth and best nodes
num_clauses, accuracy, decisions = count_rules(australian_X, australian_y, None, 5, True)

Accuracy: 0.8623188405797102
Max tree depth: 4
Num leaf nodes: 5

If/then clauses: 8
Decisions: 5


In [218]:
# Minimum clauses (ignoring accuracy)
num_clauses, accuracy, decisions = count_rules(australian_X, australian_y, 1, 2, True)

Accuracy: 0.8405797101449275
Max tree depth: 1
Num leaf nodes: 2

If/then clauses: 2
Decisions: 2


Another binary classification dataset: ilpd liver dataset

In [185]:
from sklearn.datasets import fetch_openml
liver_data = fetch_openml(name='ilpd-numeric')
liver_X = liver_data.data
liver_y = liver_data.target

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name ilpd-numeric exist. Versions may be fundamentally different, returning version 1.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [229]:
num_clauses, accuracy, decisions = count_rules(liver_X, liver_y, None, None, True)
liver_original_accuracy = accuracy
liver_original_decisions = decisions
liver_original_clauses = num_clauses

Accuracy: 0.7008547008547008
Max tree depth: 18
Num leaf nodes: 102

If/then clauses: 158
Decisions: 102


In [231]:
# Tuning depth
current_min_clauses = liver_original_clauses
current_min_decisions = liver_original_decisions
for i in range(1,19):
  num_clauses, accuracy, decisions = count_rules(liver_X, liver_y, i, 102, False)
  if (accuracy >= liver_original_accuracy):
    if (num_clauses < current_min_clauses):
      current_min_clauses = num_clauses
      print("accuracy: "  + str(accuracy))
      print("clauses: " + str(num_clauses))
      print("depth: " + str(i))
      print('\n')
    if (decisions < current_min_decisions):
      current_min_decisions = decisions
      print("accuracy: "  + str(accuracy))
      print("decisions: " + str(decisions))
      print("depth: " + str(i))

accuracy: 0.7435897435897436
clauses: 2
depth: 1


accuracy: 0.7435897435897436
decisions: 2
depth: 1


In [232]:
# Tuning nodes
current_min_clauses = liver_original_clauses
current_min_decisions = liver_original_decisions
for i in range(2,103):
  num_clauses, accuracy, decisions = count_rules(liver_X, liver_y, 18, i, False)
  if (accuracy >= liver_original_accuracy):
    if (num_clauses < current_min_clauses):
      current_min_clauses = num_clauses
      print("accuracy: "  + str(accuracy))
      print("clauses: " + str(num_clauses))
      print("nodes: " + str(i))
      print('\n')
    if (decisions < current_min_decisions):
      current_min_decisions = decisions
      print("accuracy: "  + str(accuracy))
      print("decisions: " + str(decisions))
      print("nodes: " + str(i))

accuracy: 0.7435897435897436
clauses: 2
nodes: 2


accuracy: 0.7435897435897436
decisions: 2
nodes: 2


In [233]:
# Tune both (FINAL RESULT and minimum possible clauses)
num_clauses, accuracy, decisions = count_rules(liver_X, liver_y, 1, 2, True)

Accuracy: 0.7435897435897436
Max tree depth: 1
Num leaf nodes: 2

If/then clauses: 2
Decisions: 2


### Part C

In [71]:
# 6c: Random dataset
rand_X = np.random.rand(500, 2)
rand_y = np.random.randint(0, 2, size=500)

In [79]:
# Original
num_clauses, accuracy, decisions = count_rules(rand_X, rand_y, None, None, True)
rand_original_accuracy = accuracy
rand_original_decisions = decisions
rand_original_clauses = num_clauses

Accuracy: 0.51
Max tree depth: 20
Num leaf nodes: 144

If/then clauses: 206
Decisions: 144


In [85]:
# Tuning depth
current_min_clauses = rand_original_clauses
current_min_decisions = rand_original_decisions
for i in range(1,21):
  num_clauses, accuracy, decisions = count_rules(rand_X, rand_y, i, 144, False)
  if (accuracy >= rand_original_accuracy):
    if (num_clauses < current_min_clauses):
      current_min_clauses = num_clauses
      print("accuracy: "  + str(accuracy))
      print("clauses: " + str(num_clauses))
      print("depth: " + str(i))
      print('\n')
    if (decisions < current_min_decisions):
      current_min_decisions = decisions
      print("accuracy: "  + str(accuracy))
      print("decisions: " + str(decisions))
      print("depth: " + str(i))

accuracy: 0.52
clauses: 58
depth: 6


accuracy: 0.52
decisions: 30
depth: 6


In [88]:
# Tuning nodes
current_min_clauses = rand_original_clauses
current_min_decisions = rand_original_decisions
for i in range(2,145):
  num_clauses, accuracy, decisions = count_rules(rand_X, rand_y, 20, i, False)
  if (accuracy >= rand_original_accuracy):
    if (num_clauses < current_min_clauses):
      current_min_clauses = num_clauses
      print("accuracy: "  + str(accuracy))
      print("clauses: " + str(num_clauses))
      print("nodes: " + str(i))
      print('\n')
    if (decisions < current_min_decisions):
      current_min_decisions = decisions
      print("accuracy: "  + str(accuracy))
      print("decisions: " + str(decisions))
      print("nodes: " + str(i))

accuracy: 0.51
clauses: 154
nodes: 97


accuracy: 0.51
decisions: 97
nodes: 97


In [91]:
# Tune both (FINAL RESULT)
num_clauses, accuracy, decisions = count_rules(rand_X, rand_y, 6, 97, True)

Accuracy: 0.51
Max tree depth: 6
Num leaf nodes: 30

If/then clauses: 58
Decisions: 30


In [92]:
# Minimum clauses (ignoring accuracy)
num_clauses, accuracy, decisions = count_rules(rand_X, rand_y, 1, 2, True)

Accuracy: 0.5
Max tree depth: 1
Num leaf nodes: 2

If/then clauses: 2
Decisions: 2


In our current simulation on random data, we are training on random train dataset and testing on random test dataset (two different sets of data). In this case, you could need an infinite number of if/else statements.

If we were to instead test and train on the same dataset, we would need up to the *number of instances* of if/else statements to be able to achieve 100% accuracy (one if/else statement per instance).

## CH6 EXERCISE 3

### Part A

In [277]:
import random
import string
import lzma

# Generate a long random string
random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=1000000))
unique_letters = set(random_string)
c = len(unique_letters)
print("c =", c)

# Compress the string using LZMA
compressed_string = lzma.compress(random_string.encode())

# Calculate compression ratio
compression_ratio = len(random_string) / len(compressed_string)

print(f"Original string length: {len(random_string)} bytes")
print(f"Compressed string length: {len(compressed_string)} bytes")
print(f"Compression ratio: {compression_ratio:.2f}")

c = 62
Original string length: 1000000 bytes
Compressed string length: 760732 bytes
Compression ratio: 1.31


### Part B

The expected compression ratio should be between 1 and 1.0164 (=62/61, bound derived from c/c-1 from the book).